# 🏛️ Senior - 01. Data Governance y Calidad de Datos

**Objetivos de Aprendizaje:**
- [ ] Comprender los fundamentos de Data Governance
- [ ] Implementar frameworks de calidad de datos
- [ ] Establecer data lineage y catalogación
- [ ] Aplicar principios de DAMA-DMBOK
- [ ] Implementar validaciones automatizadas con Great Expectations
- [ ] Diseñar estrategias de metadata management

**Duración Estimada:** 150 minutos  
**Nivel de Dificultad:** Avanzado  
**Prerrequisitos:** Notebooks nivel Junior y Mid completos

---

## 🎯 ¿Qué es Data Governance?

**Data Governance** es el conjunto de procesos, políticas, estándares y métricas que aseguran el uso efectivo y eficiente de la información, habilitando a una organización a alcanzar sus objetivos.

### 🏗️ Pilares de Data Governance:

1. **Data Quality** (Calidad de Datos)
   - Precisión, completitud, consistencia
   - Validaciones y monitoreo
   - Remediación de problemas

2. **Data Security** (Seguridad)
   - Control de accesos
   - Encriptación
   - Auditoría

3. **Data Lineage** (Linaje)
   - Trazabilidad origen-destino
   - Impacto de cambios
   - Documentación automática

4. **Data Catalog** (Catálogo)
   - Inventario de activos de datos
   - Metadata management
   - Descubrimiento y búsqueda

5. **Compliance** (Cumplimiento)
   - GDPR, CCPA, HIPAA
   - Políticas de retención
   - Auditorías regulatorias

### 🌟 Beneficios:

```
✅ Confianza en los datos
✅ Reducción de riesgos
✅ Cumplimiento regulatorio
✅ Mejor toma de decisiones
✅ Eficiencia operacional
✅ Colaboración mejorada
```

## 📊 Framework DAMA-DMBOK

### Las 11 Áreas de Conocimiento:

1. **Data Governance** - Marco general
2. **Data Architecture** - Estructura y diseño
3. **Data Modeling & Design** - Modelado de datos
4. **Data Storage & Operations** - Almacenamiento y operaciones
5. **Data Security** - Seguridad
6. **Data Integration & Interoperability** - Integración
7. **Document & Content Management** - Gestión documental
8. **Reference & Master Data** - Datos maestros
9. **Data Warehousing & BI** - Almacenes y BI
10. **Metadata Management** - Gestión de metadatos
11. **Data Quality** - Calidad de datos

In [ ]:
# Imports necesarios
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import hashlib
import os
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas")
print(f"📅 Sesión iniciada: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 🔍 Dimensiones de Calidad de Datos

### Las 6 Dimensiones Críticas:

In [ ]:
class DataQualityFramework:
    """
    Framework para evaluar calidad de datos
    """
    
    def __init__(self, df: pd.DataFrame, nombre_dataset: str):
        self.df = df
        self.nombre_dataset = nombre_dataset
        self.reporte = {
            'dataset': nombre_dataset,
            'fecha_evaluacion': datetime.now().isoformat(),
            'total_registros': len(df),
            'total_columnas': len(df.columns),
            'dimensiones': {}
        }
    
    def evaluar_completitud(self):
        """
        Dimensión 1: Completitud
        ¿Los datos están completos?
        """
        print("📊 Evaluando COMPLETITUD...")
        
        total_valores = self.df.size
        valores_nulos = self.df.isnull().sum().sum()
        completitud_pct = ((total_valores - valores_nulos) / total_valores) * 100
        
        # Por columna
        completitud_cols = {}
        for col in self.df.columns:
            pct = (1 - self.df[col].isnull().sum() / len(self.df)) * 100
            completitud_cols[col] = round(pct, 2)
        
        self.reporte['dimensiones']['completitud'] = {
            'score': round(completitud_pct, 2),
            'valores_nulos': int(valores_nulos),
            'por_columna': completitud_cols,
            'aprobado': completitud_pct >= 95
        }
        
        print(f"   ✓ Score: {completitud_pct:.2f}%")
        print(f"   ✓ Valores nulos: {valores_nulos:,}")
        return completitud_pct
    
    def evaluar_unicidad(self, columnas_clave: list):
        """
        Dimensión 2: Unicidad
        ¿Los registros son únicos?
        """
        print("\n🔑 Evaluando UNICIDAD...")
        
        duplicados = self.df.duplicated(subset=columnas_clave).sum()
        unicidad_pct = ((len(self.df) - duplicados) / len(self.df)) * 100
        
        self.reporte['dimensiones']['unicidad'] = {
            'score': round(unicidad_pct, 2),
            'duplicados': int(duplicados),
            'columnas_evaluadas': columnas_clave,
            'aprobado': duplicados == 0
        }
        
        print(f"   ✓ Score: {unicidad_pct:.2f}%")
        print(f"   ✓ Duplicados: {duplicados}")
        return unicidad_pct
    
    def evaluar_validez(self, reglas: dict):
        """
        Dimensión 3: Validez
        ¿Los datos cumplen reglas de negocio?
        """
        print("\n✅ Evaluando VALIDEZ...")
        
        violaciones = {}
        total_registros = len(self.df)
        registros_validos = total_registros
        
        for regla_nombre, regla_condicion in reglas.items():
            try:
                registros_invalidos = (~self.df.eval(regla_condicion)).sum()
                violaciones[regla_nombre] = int(registros_invalidos)
                registros_validos -= registros_invalidos
                print(f"   • {regla_nombre}: {registros_invalidos} violaciones")
            except Exception as e:
                print(f"   ⚠️ Error en regla '{regla_nombre}': {e}")
        
        validez_pct = (registros_validos / total_registros) * 100
        
        self.reporte['dimensiones']['validez'] = {
            'score': round(validez_pct, 2),
            'violaciones': violaciones,
            'reglas_evaluadas': list(reglas.keys()),
            'aprobado': validez_pct >= 95
        }
        
        print(f"   ✓ Score: {validez_pct:.2f}%")
        return validez_pct
    
    def evaluar_consistencia(self, columnas_relacionadas: dict):
        """
        Dimensión 4: Consistencia
        ¿Los datos son consistentes entre sí?
        """
        print("\n🔄 Evaluando CONSISTENCIA...")
        
        inconsistencias = {}
        
        for nombre, condicion in columnas_relacionadas.items():
            try:
                count = (~self.df.eval(condicion)).sum()
                inconsistencias[nombre] = int(count)
                print(f"   • {nombre}: {count} inconsistencias")
            except Exception as e:
                print(f"   ⚠️ Error en validación '{nombre}': {e}")
        
        total_inconsistencias = sum(inconsistencias.values())
        consistencia_pct = ((len(self.df) - total_inconsistencias) / len(self.df)) * 100
        
        self.reporte['dimensiones']['consistencia'] = {
            'score': round(consistencia_pct, 2),
            'inconsistencias': inconsistencias,
            'aprobado': consistencia_pct >= 95
        }
        
        print(f"   ✓ Score: {consistencia_pct:.2f}%")
        return consistencia_pct
    
    def evaluar_exactitud(self, columna_numerica: str, rango_esperado: tuple):
        """
        Dimensión 5: Exactitud
        ¿Los valores son correctos?
        """
        print("\n🎯 Evaluando EXACTITUD...")
        
        min_val, max_val = rango_esperado
        fuera_rango = ((self.df[columna_numerica] < min_val) | 
                       (self.df[columna_numerica] > max_val)).sum()
        
        exactitud_pct = ((len(self.df) - fuera_rango) / len(self.df)) * 100
        
        self.reporte['dimensiones']['exactitud'] = {
            'score': round(exactitud_pct, 2),
            'fuera_de_rango': int(fuera_rango),
            'columna_evaluada': columna_numerica,
            'rango_esperado': rango_esperado,
            'aprobado': fuera_rango == 0
        }
        
        print(f"   ✓ Score: {exactitud_pct:.2f}%")
        print(f"   ✓ Fuera de rango: {fuera_rango}")
        return exactitud_pct
    
    def evaluar_actualidad(self, columna_fecha: str, dias_maximos: int):
        """
        Dimensión 6: Actualidad
        ¿Los datos son recientes?
        """
        print("\n📅 Evaluando ACTUALIDAD...")
        
        hoy = pd.Timestamp.now()
        self.df[columna_fecha] = pd.to_datetime(self.df[columna_fecha])
        antiguos = (hoy - self.df[columna_fecha]) > timedelta(days=dias_maximos)
        registros_antiguos = antiguos.sum()
        
        actualidad_pct = ((len(self.df) - registros_antiguos) / len(self.df)) * 100
        
        self.reporte['dimensiones']['actualidad'] = {
            'score': round(actualidad_pct, 2),
            'registros_antiguos': int(registros_antiguos),
            'dias_maximos': dias_maximos,
            'aprobado': actualidad_pct >= 80
        }
        
        print(f"   ✓ Score: {actualidad_pct:.2f}%")
        print(f"   ✓ Registros antiguos: {registros_antiguos}")
        return actualidad_pct
    
    def generar_reporte_completo(self):
        """
        Genera reporte ejecutivo de calidad
        """
        print("\n" + "="*60)
        print(f"📊 REPORTE DE CALIDAD DE DATOS: {self.nombre_dataset}")
        print("="*60)
        
        # Calcular score general
        scores = [d['score'] for d in self.reporte['dimensiones'].values() if 'score' in d]
        score_general = sum(scores) / len(scores) if scores else 0
        
        self.reporte['score_general'] = round(score_general, 2)
        self.reporte['nivel_calidad'] = self._clasificar_calidad(score_general)
        
        print(f"\n🎯 Score General: {score_general:.2f}%")
        print(f"📈 Nivel de Calidad: {self.reporte['nivel_calidad']}")
        print(f"\n📋 Detalles por Dimensión:")
        
        for dimension, datos in self.reporte['dimensiones'].items():
            status = "✅" if datos.get('aprobado', False) else "❌"
            print(f"   {status} {dimension.upper()}: {datos.get('score', 'N/A')}%")
        
        # Guardar reporte
        output_path = f"../datasets/processed/reporte_calidad_{self.nombre_dataset}.json"
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(self.reporte, f, indent=2, ensure_ascii=False)
        
        print(f"\n💾 Reporte guardado en: {output_path}")
        return self.reporte
    
    @staticmethod
    def _clasificar_calidad(score):
        """Clasifica el nivel de calidad según el score"""
        if score >= 95:
            return "⭐⭐⭐ EXCELENTE"
        elif score >= 85:
            return "⭐⭐ BUENO"
        elif score >= 70:
            return "⭐ ACEPTABLE"
        else:
            return "❌ CRÍTICO"

print("✅ DataQualityFramework definido")

## 🧪 Caso Práctico: Evaluación de Calidad

Vamos a evaluar un dataset de ventas:

In [ ]:
# Crear dataset de ejemplo con problemas de calidad intencionales
np.random.seed(42)

n_registros = 1000

df_ventas = pd.DataFrame({
    'venta_id': range(1, n_registros + 1),
    'cliente_id': np.random.randint(1, 200, n_registros),
    'producto': np.random.choice(['Laptop', 'Mouse', 'Teclado', 'Monitor', None], n_registros),
    'cantidad': np.random.randint(-5, 50, n_registros),  # Algunas cantidades negativas
    'precio_unitario': np.random.uniform(10, 2000, n_registros),
    'descuento_pct': np.random.uniform(0, 100, n_registros),  # Algunos > 100%
    'total': np.random.uniform(0, 5000, n_registros),
    'fecha_venta': pd.date_range(start='2023-01-01', periods=n_registros, freq='H'),
    'estado': np.random.choice(['Completada', 'Pendiente', 'Cancelada', None], n_registros)
})

# Introducir duplicados intencionales
df_ventas = pd.concat([df_ventas, df_ventas.iloc[:10]], ignore_index=True)

# Introducir algunos nulls adicionales
df_ventas.loc[np.random.choice(df_ventas.index, 50), 'precio_unitario'] = np.nan

print(f"📊 Dataset creado: {len(df_ventas)} registros, {len(df_ventas.columns)} columnas")
print(f"\n🔍 Primeras filas:")
print(df_ventas.head())
print(f"\n📈 Info del dataset:")
print(df_ventas.info())

In [ ]:
# Inicializar framework
dq = DataQualityFramework(df_ventas, 'ventas_ecommerce')

# Evaluar todas las dimensiones
print("🚀 Iniciando evaluación de calidad...\n")

# 1. Completitud
dq.evaluar_completitud()

# 2. Unicidad
dq.evaluar_unicidad(columnas_clave=['venta_id'])

# 3. Validez
reglas_negocio = {
    'cantidad_positiva': 'cantidad > 0',
    'descuento_valido': 'descuento_pct >= 0 and descuento_pct <= 100',
    'total_positivo': 'total > 0'
}
dq.evaluar_validez(reglas_negocio)

# 4. Consistencia
relaciones = {
    'total_coherente': 'total >= (precio_unitario * cantidad * (1 - descuento_pct/100)) * 0.95',
}
dq.evaluar_consistencia(relaciones)

# 5. Exactitud
dq.evaluar_exactitud('precio_unitario', (5, 3000))

# 6. Actualidad
dq.evaluar_actualidad('fecha_venta', dias_maximos=365)

# Generar reporte completo
reporte = dq.generar_reporte_completo()

## 📋 Data Lineage - Trazabilidad de Datos

Implementación de un sistema básico de lineage:

In [ ]:
class DataLineageTracker:
    """
    Sistema para rastrear el linaje de datos
    """
    
    def __init__(self):
        self.linaje = []
        self.grafo = {}
    
    def registrar_transformacion(self, 
                                 origen: str, 
                                 destino: str, 
                                 transformacion: str,
                                 metadata: dict = None):
        """
        Registra una transformación de datos
        """
        evento = {
            'timestamp': datetime.now().isoformat(),
            'origen': origen,
            'destino': destino,
            'transformacion': transformacion,
            'metadata': metadata or {},
            'id': hashlib.md5(f"{origen}{destino}{datetime.now()}".encode()).hexdigest()[:8]
        }
        
        self.linaje.append(evento)
        
        # Construir grafo
        if origen not in self.grafo:
            self.grafo[origen] = []
        self.grafo[origen].append(destino)
        
        print(f"📝 Registrado: {origen} → {destino} ({transformacion})")
        return evento['id']
    
    def obtener_linaje_completo(self, entidad: str, direccion='downstream'):
        """
        Obtiene el linaje completo de una entidad
        """
        if direccion == 'downstream':
            # Hacia adelante (downstream)
            return self._traverse_downstream(entidad)
        else:
            # Hacia atrás (upstream)
            return self._traverse_upstream(entidad)
    
    def _traverse_downstream(self, nodo, visitados=None):
        if visitados is None:
            visitados = set()
        
        if nodo in visitados:
            return []
        
        visitados.add(nodo)
        camino = [nodo]
        
        if nodo in self.grafo:
            for hijo in self.grafo[nodo]:
                camino.extend(self._traverse_downstream(hijo, visitados))
        
        return camino
    
    def _traverse_upstream(self, nodo):
        # Invertir el grafo para upstream
        grafo_invertido = {}
        for origen, destinos in self.grafo.items():
            for destino in destinos:
                if destino not in grafo_invertido:
                    grafo_invertido[destino] = []
                grafo_invertido[destino].append(origen)
        
        return self._traverse_downstream(nodo, set())
    
    def visualizar_linaje(self):
        """
        Genera visualización simple del linaje
        """
        print("\n📊 MAPA DE LINAJE DE DATOS")
        print("="*50)
        
        for origen, destinos in self.grafo.items():
            print(f"\n🔹 {origen}")
            for destino in destinos:
                # Encontrar transformación
                trans = next((e for e in self.linaje 
                            if e['origen'] == origen and e['destino'] == destino), {})
                print(f"   └─→ {destino} [{trans.get('transformacion', 'N/A')}]")
    
    def exportar_linaje(self, ruta_salida: str):
        """
        Exporta linaje a archivo JSON
        """
        os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)
        
        with open(ruta_salida, 'w', encoding='utf-8') as f:
            json.dump({
                'eventos': self.linaje,
                'grafo': self.grafo,
                'generado': datetime.now().isoformat()
            }, f, indent=2, ensure_ascii=False)
        
        print(f"\n💾 Linaje exportado a: {ruta_salida}")

# Ejemplo de uso
tracker = DataLineageTracker()

print("🔍 Simulando pipeline con lineage tracking:\n")

# Registrar transformaciones
tracker.registrar_transformacion(
    'raw_ventas_csv',
    'stage_ventas',
    'extraccion',
    {'filas': 1000, 'columnas': 9}
)

tracker.registrar_transformacion(
    'stage_ventas',
    'clean_ventas',
    'limpieza',
    {'nulos_removidos': 50, 'duplicados_removidos': 10}
)

tracker.registrar_transformacion(
    'clean_ventas',
    'agg_ventas_diarias',
    'agregacion',
    {'grupo_por': 'fecha', 'metricas': ['sum', 'count', 'avg']}
)

tracker.registrar_transformacion(
    'clean_ventas',
    'ventas_por_producto',
    'agregacion',
    {'grupo_por': 'producto'}
)

tracker.registrar_transformacion(
    'agg_ventas_diarias',
    'dashboard_ventas',
    'visualizacion',
    {'tipo': 'time_series'}
)

# Visualizar
tracker.visualizar_linaje()

# Exportar
tracker.exportar_linaje('../datasets/processed/data_lineage.json')

# Consultar linaje
print("\n🔎 Linaje downstream de 'clean_ventas':")
downstream = tracker.obtener_linaje_completo('clean_ventas', 'downstream')
print(f"   {' → '.join(downstream)}")

## 🎓 Resumen y Mejores Prácticas

### ✅ Principios Clave de Data Governance:

1. **Proactivo, no Reactivo**
   - Validaciones automáticas en cada etapa
   - Prevención vs corrección

2. **Cultura de Calidad**
   - Responsabilidad compartida
   - Data stewards claramente definidos

3. **Automatización**
   - Tests automatizados
   - Monitoreo continuo
   - Alertas proactivas

4. **Documentación**
   - Lineage completo
   - Metadata actualizado
   - Políticas claras

5. **Balance**
   - Gobernanza sin burocracia
   - Flexibilidad con control

### 🔜 Próximos Temas:

- Implementación de Data Catalogs
- Master Data Management (MDM)
- Privacy by Design y PII handling
- Compliance automation (GDPR, CCPA)
- Cost optimization y FinOps

---

**¡Has dominado los fundamentos de Data Governance!** 🏆